### Rename Data Frame Columns


In [2]:
def guess_column_names(columnname):
    ''' An attempt to standardize and rename column headers for manipulation later.
        Input: column name (String)
        Output: Corrected name (String)
    '''
    from fuzzywuzzy import fuzz
    from fuzzywuzzy import process 
    
    #list of possible "correct" column headers 
    correct_headers = ['First Name', 'Last Name','Fullname','Student Name','Job Title', 'Title','ID',
                       'Institution','School','Company','Company Name1','Company Name2','Organization Name','Department','Division',
                       'Email Address','Street Address','Street 1','Dorm Address 1','Dorm Address 2','Dorm Address 3',
                       'Dorm Address 4','Address 1','Street 2','Address 2','Address','Street 3','Address 3','Street 4','Address 4',
                       'Work Street 1','Work Street 2','Work Street 2','Work Street 3','Work Street 4',
                       'Zipcode','Home Zipcode','Work_City','Dorm Postalplus4','HOME_FOREIGN_CITYZIP','WORK_FOREIGN_CITYZIP','Work_State','Work_Country',
                       'Postal','City','County','State','Country']
    
    # if column is exact match return name
    if columnname in correct_headers:  # might want to make this a dict for O(1) lookups
        return columnname#, 100
    
    # if column name is longer than 20 characters, return best quess based on last 15 characters
    if len(columnname) > 20:
        new_name, score = process.extractOne(columnname[-15:], correct_headers) 
        return new_name#, score
    
    # for all others, 
    else:
        new_name, score = process.extractOne(columnname, correct_headers)
        
    #if score > 80, return new_name 
    if score < 80:
        # returns orginal name if match is bad
        return columnname#, score
    else:
        return new_name#, score

In [3]:
def unique_columns(df_columns):
    ''' Columns with same name get numbered to avoid duplication
        Input: list of columns
        Output: list of columns with appended numbers 
    '''
    
    seen = set()
    for col in df_columns:
        append = 1 
        newitem = col

        while newitem in seen:
            append += 1
            newitem = "{} {}".format(col, append)

        yield newitem
        seen.add(newitem)

In [28]:
def drop_columns(df):
    ''' Columns of the dataframe are dropped if they are not in the list below '''
    
    # If names don't match this list drop them from dataframe
    headers = ['First Name','Last Name','Fullname','Title',
               'Company','Department',
               'Address 1','Address 2',
               'City','State','Zipcode','Country']
    
    cols = [col for col in df.columns if col not in headers]
    #print cols
    df.drop(cols, axis=1, inplace=True)
    #print 'Drop: ',df.columns
    return df

In [5]:
# reorder columns
def reorder_columns(dataframe, seq):
    '''Takes a dataframe and a sequence of columns names,
       returns dataframe with seq as first columns. If seq contains columns 
       that aren't in the dataframe then the columns are created with Nan values. 
    '''
    import numpy as np
    cols = seq[:]
    for x in dataframe.columns:
        if x not in cols:
             cols.append(x)
            
    for x in cols:
        if x not in dataframe.columns: #If column from seq is not in df, 
            dataframe[x]= np.nan       #create a new column filled with Nan
    
    return dataframe[cols]

In [35]:
def renameCol(df):
    
    import pandas as pd
    
    # List of New "corrected" column names
    new_col = [guess_column_names(col) for col in df.columns]
    
    #Set df columns equal to "corrected" columns 
    df.columns = new_col
    #print 'Corrected: ',df.columns
    
    # Rename Columns 
    df.rename(columns={'School':'Department',
                       'Institution':'Company',
                       'Organization':'Company',
                       'Organization Name':'Company',
                       'Street Address':'Street 1',
                       'Division':'Department',
                       'Postal':'Zipcode',
                       'Zip':'Zipcode',
                       'Home Zipcode':'Zipcode',
                       'Street 1':'Address 1',
                       'Street 2':'Address 2',
                       'Street 3':'Address 3',
                       'PERSON_NAME':'Fullname',
                       'Student Name':'Fullname'  #???
                       
                      }, inplace=True)
        
    #print 'Rename: ',df.columns
    
    #Call name split here
    
    
    
    # Create unique versions of Columns to avoid issues with pandas 
    df.columns = list(unique_columns(df.columns))
    #print 'Unique: ',df.columns
    
    # Drop columns not needed 
    df = drop_columns(df)
    #print 'Drop: ',df.columns
    
    # Reorder Columns in df 
    df = reorder_columns(df,['First Name','Last Name','Fullname','Title','Company','Department','Address 1','Address 2','City','State','Zipcode','Country'])
    #print 'Reorder: ', df.columns
    
    return df

In [36]:
# Import data to dataframe for testing 
import pandas as pd
df = pd.read_excel('data/2159StudentsAddFERPAisN.XLSX')
df1 = pd.read_excel('data/Business and Deans combined list 10-09-2015.xlsx')

In [39]:
#
df = renameCol(df)
df1 = renameCol(df1)

In [45]:
# Concate each dataframe 
results = pd.concat([df,df1])